# 1. Load Raw Data (with Sentences and Ground Truths)

In [6]:
import os
import pandas as pd

file_name = 'raw_data_with_gt.pkl'
data_path = os.path.join('..', 'data', file_name )
print("Raw data path:", data_path)
data_df = pd.read_pickle(data_path)
data_df.head(1)

Raw data path: ..\data\raw_data_with_gt.pkl


,id,text,chunk,ner_label,label_count
0,50,"Unfortunately , I think itis unlikelythathis s...","['he', 'over 9 to 12 months', 'he', 'he']","[Gender, RelativeDate, Gender, Gender]","{'Gender': 3, 'RelativeDate': 1}"


# 2. Task Selection

**Prediction Tasks**
1. 'Organ_Component'
   <br> *Evaluated for* 
    - External_body_part_or_region
    - Internal_organ_or_component
2. 'Gender'
3. 'Procedure_Treatment'
    <br> *Evaluated for* 
    - Procedure
    - Treatment
4. 'Problem' (19 Entities merged under Problem Type)
5. 'Medicine' (Drug_BrandName, Drug_Ingredient, Strength, Form, Dosage )
<br> In this evaluation case partial matches are considered as full_match
6. 'Drug'
    <br> *Evaluated for* 
    - Drug (Drug_BrandName, Drug_Ingredient)
    - Drug_Related (Strength, Form, Dosage)
7. 'Test'
8. 'Test_TestResults' (Test, Test Results)
    <br> *Evaluated for* 
    - Test
    - Test_Result

In [7]:
from modules.ProcessRawData import ProcessData
from modules.ProcessPredData import corrected_json, get_list_of_entities
import json

# read tasks from file
with open("./sources/tasks_list.json", "r") as f:
    data = json.load(f)

# extract problem_entity_list and task_list
task_list = data["task_list"]
eval_options_dict = data["eval_options_dict"]

print("task_list:", task_list.keys())
print("eval_options_dict:",eval_options_dict)

task_list: dict_keys(['Deid', 'Organ_Component', 'Gender', 'Procedure_Treatment', 'Problem', 'Medicine', 'Drug', 'Test', 'Test_TestResult', 'Oncological'])
eval_options_dict: {'External_body_part_or_region': {'selected_entity_prediction': ['External_body_part_or_region'], 'selected_entity_gt': ['External_body_part_or_region'], 'gt_type_dict': {'External_body_part_or_region': 'External_body_part_or_region'}}, 'Internal_organ_or_component': {'selected_entity_prediction': ['Internal_organ_or_component'], 'selected_entity_gt': ['Internal_organ_or_component'], 'gt_type_dict': {'Internal_organ_or_component': 'Internal_organ_or_component'}}, 'Gender': {'selected_entity_prediction': ['Gender'], 'selected_entity_gt': ['Gender'], 'gt_type_dict': {'Gender': 'Gender'}}, 'Procedure': {'selected_entity_prediction': ['Procedure'], 'selected_entity_gt': ['Procedure'], 'gt_type_dict': {'Procedure': 'Procedure'}}, 'Treatment': {'selected_entity_prediction': ['Treatment'], 'selected_entity_gt': ['Treatme

## 2.1 Select Task to Get Predictions

In [22]:
## !!! USER SELECTION NECESSARY in this cell

# # select entity for evaluation
# entity_options = list(task_list.keys())
# entity_prompt = "Please select an entity for prediction generatio (options are: {}): ".format(", ".join(entity_options))
# entity_under_test = input(entity_prompt)

# # check if the user input is valid
# while entity_under_test not in entity_options:
#     print("Invalid input. Please try again.")
#     entity_under_test = input(entity_prompt)

# # continue with evaluation using selected entity_under_test
# print("Predictions will be generated for the entities:", entity_under_test)

import ipywidgets as widgets

# select entity for evaluation
entity_options = list(task_list.keys())

def assign_entity(entity):
    global entity_under_test
    entity_under_test = entity
    print("Predictions will be generated for the entities:", entity_under_test)

# create the dropdown menu
dropdown = widgets.Dropdown(options=entity_options)

# create the button
button = widgets.Button(description="Approve")

# define the callback function for the button
def on_button_click(button):
    assign_entity(dropdown.value)

# register the callback function with the button
button.on_click(on_button_click)

# display the dropdown menu and the button
display(dropdown)
display(button)


Dropdown(options=('Deid', 'Organ_Component', 'Gender', 'Procedure_Treatment', 'Problem', 'Medicine', 'Drug', '…

Button(description='Approve', style=ButtonStyle())

Predictions will be generated for the entities: Oncological
Predictions will be generated for the entities: Oncological


# 3. Prepare Data for Prediction

In [32]:
# filter gt data for prediction based on selected entity under test and entity counts
covered_gt_label_list = task_list[entity_under_test]['covered_gt_label_list']
count_filter = task_list[entity_under_test]['count_filter']

prediction_source = "ChatGPT"

# process gt data for prediction and evaluation
process_gt_data = ProcessData(
    covered_gt_label_list, 
    count_filter, 
    data_path,
    prediction_source
)

processed_gt_df = process_gt_data.get_final_df()

2023-11-06 02:22:55,165 - ProcessRawData - INFO - main df shape: (849, 5)
2023-11-06 02:22:55,167 - ProcessRawData - INFO - filtered df shape: (0, 6)
2023-11-06 02:22:55,168 - ProcessRawData - INFO - final df shape: (0, 8)


Processing main csv...


# 4. GPT Predictions 

## 4.1 Load Prompt and Initiate Entity Extractor

In [33]:
from modules import NerExtraction
prompt_file_path = os.path.join('.', 'prompts', task_list[entity_under_test]['prompt_file'] )
print(f"Prompt for [{entity_under_test}] extraction task: ({prompt_file_path})")

with open(prompt_file_path, 'r') as file:
    print("PROMPT:\n")
    for line in file:
        print(line)
    
ner_extraction = NerExtraction.ChatGPTNER(prompt_file_path)

Prompt for [Oncological] extraction task: (.\prompts\11_02_Oncological.txt)
PROMPT:



You are a highly experienced and skilled medical annotator who have been working on medical texts to label medical entities.



I will provide you some entity types with sample chunks and I want you to find similar entities from given texts and label them with right entity types.



-  Entity Type: Oncological

    Instruction: include all the cancer, tumor or metastasis related extractions mentioned in the document, of the patient or someone else.



    Examples:

    a) given sample sentence:

    His mother was diagnosed with colon cancer in her 50s, but she died of cancer of the esophagus at age 86.

    Oncological Entities in above given text: colon cancer, cancer of the esophagus



    b) given sample sentence:

    She was diagnosed with pseudomyxoma peritonei in 1994.

    Oncological Entities in above given text: pseudomyxoma peritonei



I want you to extract all Oncological type entitie

## 4.2 Get Prediction

In [34]:
# !!! Run without any changes

from modules import BatchProcessing
from modules.ProcessPredData import corrected_json, get_list_of_entities
import datetime

# Assing auto name to save prediction data as csv and excel
now = datetime.datetime.now()
file_name = f"{entity_under_test}_preds_{now.strftime('%m%d_%H%M')}"
processed_data_path = os.path.join('.', 'processed_data', file_name)

print(processed_data_path)
batch_processor = BatchProcessing.ProcessBatch(
    processed_gt_df,
    ner_extraction,
    corrected_json,
    get_list_of_entities,
    processed_data_path
)

results_df = batch_processor.do_processing()

.\processed_data\Oncological_preds_1106_0222
Given dataframe shape (0, 8)
Getting predictions from API started...


KeyError: 'ground_truth_list'

# 5. Evaluation

## 5.1 Selection of entity for evaluation

In [18]:
###!!! USER INPUT MIGHT BE NECESSARY if there are multiple entites for evaluation under given task
## selection of entity type for benchmark

# entity_list_for_benchmark = data["entity_for_benchmark"][entity_under_test]
# print("entity_list_for_benchmark:", entity_list_for_benchmark)
# if len(entity_list_for_benchmark) == 1:
#     entity_for_benchmark = entity_list_for_benchmark[0]
#     print("Benchmarks will be evaluated for the entity:", entity_for_benchmark)
# else:
#     entity_prompt = "Please select an entity for evaluation (options are: {}): ".format(", ".join(entity_list_for_benchmark))
#     entity_for_benchmark = input(entity_prompt)
#     print("Benchmarks will be evaluated for the entity:", entity_for_benchmark)

import ipywidgets as widgets

# selection of entity type for benchmark
entity_list_for_benchmark = data["entity_for_benchmark"][entity_under_test]
print("entity_list_for_benchmark:", entity_list_for_benchmark)

if len(entity_list_for_benchmark) == 1:
    entity_for_benchmark = entity_list_for_benchmark[0]
    print("Benchmarks will be evaluated for the entity:", entity_for_benchmark)
else:
    def assign_entity(entity):
        global entity_for_benchmark
        entity_for_benchmark = entity
        print("Benchmarks will be evaluated for the entity:", entity_for_benchmark)
    
    # create the dropdown menu
    dropdown = widgets.Dropdown(options=entity_list_for_benchmark)

    # create the button
    button = widgets.Button(description="Approve")

    # define the callback function for the button
    def on_button_click(button):
        assign_entity(dropdown.value)

    # register the callback function with the button
    button.on_click(on_button_click)

    # display the dropdown menu and the button
    display(dropdown)
    display(button)


entity_list_for_benchmark: ['Test', 'Test_Result']


Dropdown(options=('Test', 'Test_Result'), value='Test')

Button(description='Approve', style=ButtonStyle())

Benchmarks will be evaluated for the entity: Test
Benchmarks will be evaluated for the entity: Test_Result


## 5.2 Run Evaluation 

In [20]:
# No user interaction needed, just run the cell
from modules import Evaluation
import datetime
now = datetime.datetime.now()

eval_file_to_save = entity_for_benchmark + "_" + prediction_source + f"_eval_{now.strftime('%m%d_%H%M')}"

selected_entity_prediction = eval_options_dict[entity_for_benchmark]["selected_entity_prediction"]
selected_entity_gt = eval_options_dict[entity_for_benchmark]["selected_entity_gt"]
gt_type_dict = eval_options_dict[entity_for_benchmark]["gt_type_dict"]

file_path__to_read_prediction = f"{processed_data_path}.csv"
# alternative to reading prediction results from file 
# dataframe = results_df !!! make >> file_path__to_read_prediction = None 

evaluator = Evaluation.Evaluate(
    file_path=file_path__to_read_prediction, 
    dataframe=None, 
    prediction_source=prediction_source
)

eval_results = evaluator.get_match_counts(
    selected_entity_prediction, 
    selected_entity_gt, 
    eval_file_to_save,
    gt_type_dict
)

# create folder if it doesn't exist
if not os.path.exists('eval_results'):
    os.makedirs('eval_results')
    
# write result to JSON file
with open('eval_results/eval_result.json', 'a') as f:
    json.dump(eval_results, f)
    print("Results appended to file: eval_results/eval_result.json")
    
eval_results

Evaluation results saved as ./eval_results/Test_Result_ChatGPT_eval_1106_0220.xlsx
Results appended to file: eval_results/eval_result.json


{'version': 'Test_Result_ChatGPT_eval_1106_0220',
 'selected_entity_prediction': ['Test_Result'],
 'selected_entity_gt': ['Test_Result'],
 'full_match': 277,
 'accuracy_full_match': 0.76,
 'partial_match': 30,
 'accuracy_partial_match': 0.84,
 'no_match': 59,
 'gt_count': 366,
 'fp_count': 42}